In [1]:
import pandas_ta as ta
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import yfinance as yf
import matplotlib.pyplot as plt

In [19]:
# !pip install backtesting

In [3]:
import backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
# backtesting.set_bokeh_output(notebook=False) # fix warning from lack of JS

In [16]:
# Simple Moving  Average
def SMA(values, n):
    '''
    Return Simple moving Average of 'values',at each step takinginto account 'n' previous values.
    '''
    return pd.Series(values).rolling(n).mean()

# SMA Cross Strategy class
class SmaCross(Strategy):
    n1 = 25
    n2 = 30

    def init(self):
        # precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        # If sma1 crosses above sma2, close any existing trade
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [17]:
data = yf.download('BTC-USD')
# data = pd.read_csv("BTCUSD_M15_TEST.csv",index_col = 'Date')
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...
2024-10-18,67419.109375,68969.750000,67177.820312,68418.789062,68418.789062,36857165014
2024-10-19,68418.976562,68668.007812,68024.640625,68362.734375,68362.734375,14443497908
2024-10-20,68364.179688,69359.007812,68105.718750,69001.703125,69001.703125,18975847518


In [18]:
# data.index = pd.to_datetime(data.index, format='mixed')
# type(data.index)

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3689 entries, 2014-09-17 to 2024-10-22
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       3689 non-null   float64
 1   High       3689 non-null   float64
 2   Low        3689 non-null   float64
 3   Close      3689 non-null   float64
 4   Adj Close  3689 non-null   float64
 5   Volume     3689 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 201.7 KB


In [20]:
bt = Backtest(data, SmaCross)
stat = bt.run()
stat

C:\Users\The Geek Dev\AppData\Local\Temp\ipykernel_10808\60719262.py:1: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, SmaCross)


Start                     2014-09-17 00:00:00
End                       2024-10-22 00:00:00
Duration                   3688 days 00:00:00
Exposure Time [%]                   98.807265
Equity Final [$]                 544769.66391
Equity Peak [$]                 756394.003754
Return [%]                        5347.696639
Buy & Hold Return [%]            14504.461881
Return (Ann.) [%]                   48.520284
Volatility (Ann.) [%]              101.616274
Sharpe Ratio                         0.477485
Sortino Ratio                        1.157888
Calmar Ratio                         0.669015
Max. Drawdown [%]                  -72.524993
Avg. Drawdown [%]                  -11.825992
Max. Drawdown Duration      566 days 00:00:00
Avg. Drawdown Duration       60 days 00:00:00
# Trades                                  167
Win Rate [%]                        49.101796
Best Trade [%]                     265.605778
Worst Trade [%]                    -41.624433
Avg. Trade [%]                    

In [21]:
bt.plot()
stat['_trades']

C:\Users\The Geek Dev\AppData\Roaming\Python\Python312\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\The Geek Dev\AppData\Roaming\Python\Python312\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\The Geek Dev\AppData\Roaming\Python\Python312\site-packages\backtesting\_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
C:\Users\The Geek Dev\AppData\Roaming\Python\Python312\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\The G

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,28,44,51,345.009003,349.817993,134.651733,0.013939,2014-10-31,2014-11-07,7 days
1,-28,51,65,349.817993,357.878998,-225.708130,-0.023043,2014-11-07,2014-11-21,14 days
2,27,65,83,357.878998,361.894989,108.431763,0.011222,2014-11-21,2014-12-09,18 days
3,-27,83,147,361.894989,219.731995,3838.400848,0.392829,2014-12-09,2015-02-11,64 days
4,63,147,156,219.731995,240.251007,1292.697784,0.093382,2015-02-11,2015-02-20,9 days
...,...,...,...,...,...,...,...,...,...,...
162,9,3646,3647,57020.097656,57650.289062,5671.722656,0.011052,2024-09-10,2024-09-11,1 days
163,-9,3647,3649,57650.289062,58130.324219,-4320.316406,-0.008327,2024-09-11,2024-09-13,2 days
164,8,3649,3651,58130.324219,60000.726562,14963.218750,0.032176,2024-09-13,2024-09-15,2 days
165,-8,3651,3662,60000.726562,63138.546875,-25102.562500,-0.052296,2024-09-15,2024-09-26,11 days


## Optimizing Strategies in Backtesting...

In [14]:
param_grid = {'n1':range(10,30,5), 'n2':range(20,50,5)}
# Run teh optimization
res = bt.optimize(**param_grid)
# Print the best results adn teh parameters that lead to these results
print("Best Results: ", res['Return [%]'])
print("Parameters for best result: ", res['_strategy'])

  0%|          | 0/4 [00:00<?, ?it/s]

Best Results:  5347.675389099121
Parameters for best result:  SmaCross(n1=25,n2=30)


In [15]:
res

Start                     2014-09-17 00:00:00
End                       2024-10-22 00:00:00
Duration                   3688 days 00:00:00
Exposure Time [%]                   98.807265
Equity Final [$]                 544767.53891
Equity Peak [$]                 756394.003754
Return [%]                        5347.675389
Buy & Hold Return [%]            14504.519962
Return (Ann.) [%]                   48.520227
Volatility (Ann.) [%]              101.616234
Sharpe Ratio                         0.477485
Sortino Ratio                        1.157887
Calmar Ratio                         0.669014
Max. Drawdown [%]                  -72.524993
Avg. Drawdown [%]                  -11.825992
Max. Drawdown Duration      566 days 00:00:00
Avg. Drawdown Duration       60 days 00:00:00
# Trades                                  167
Win Rate [%]                        49.101796
Best Trade [%]                     265.605778
Worst Trade [%]                    -41.624433
Avg. Trade [%]                    

In [22]:
res.index

Index(['Start', 'End', 'Duration', 'Exposure Time [%]', 'Equity Final [$]',
       'Equity Peak [$]', 'Return [%]', 'Buy & Hold Return [%]',
       'Return (Ann.) [%]', 'Volatility (Ann.) [%]', 'Sharpe Ratio',
       'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]',
       'Avg. Drawdown [%]', 'Max. Drawdown Duration', 'Avg. Drawdown Duration',
       '# Trades', 'Win Rate [%]', 'Best Trade [%]', 'Worst Trade [%]',
       'Avg. Trade [%]', 'Max. Trade Duration', 'Avg. Trade Duration',
       'Profit Factor', 'Expectancy [%]', 'SQN', '_strategy', '_equity_curve',
       '_trades'],
      dtype='object')

In [24]:
res['_trades'] # access all the trades that were taken

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,28,44,51,345.009003,349.817993,134.651733,0.013939,2014-10-31,2014-11-07,7 days
1,-28,51,65,349.817993,357.878998,-225.708130,-0.023043,2014-11-07,2014-11-21,14 days
2,27,65,83,357.878998,361.894989,108.431763,0.011222,2014-11-21,2014-12-09,18 days
3,-27,83,147,361.894989,219.731995,3838.400848,0.392829,2014-12-09,2015-02-11,64 days
4,63,147,156,219.731995,240.251007,1292.697784,0.093382,2015-02-11,2015-02-20,9 days
...,...,...,...,...,...,...,...,...,...,...
162,9,3646,3647,57020.097656,57650.289062,5671.722656,0.011052,2024-09-10,2024-09-11,1 days
163,-9,3647,3649,57650.289062,58130.324219,-4320.316406,-0.008327,2024-09-11,2024-09-13,2 days
164,8,3649,3651,58130.324219,60000.726562,14963.218750,0.032176,2024-09-13,2024-09-15,2 days
165,-8,3651,3662,60000.726562,63138.546875,-25102.562500,-0.052296,2024-09-15,2024-09-26,11 days


In [25]:
res['_equity_curve']

,Equity,DrawdownPct,DrawdownDuration
2014-09-17,10000.00000,0.000000,NaT
2014-09-18,10000.00000,0.000000,NaT
2014-09-19,10000.00000,0.000000,NaT
2014-09-20,10000.00000,0.000000,NaT
2014-09-21,10000.00000,0.000000,NaT
...,...,...,...
2024-10-18,548708.47641,0.274573,NaT
2024-10-19,548260.03891,0.275166,NaT
2024-10-20,553371.78891,0.268408,NaT
2024-10-21,540300.97641,0.285688,NaT


# __Backtesting Quality Ratios__
* `sharpe ratio`, `sortino ratio` and the `kalmar ration`
1. Sharpe Ratio - measures the perfomance against a risk-free asset.
   * Formula: (Portfolio returns - Risk-free returns) / Standard deviation.
   * Indicates higher returns for a given level of risk.
   * How our Returns are going to fluctuate over time
   * We need to have steady returns with minimal standard deviations
2. Sortino Ratio
   * Differentiates harmful volatility using downside deviation.
   * Formula: [Actual or expected return - Risk-free rate) / Standard deviation of downside.
   * Focuses on minimizing large losses
3. Calmar Ratio
   * Compares average annual return to maximum drawdown risk.
   * Formula: Annualized portfolio return / Maximum drawdown.
   * Indicates better risk-adjusted perfomance
   The returns of a portfolio if the reutrns were computed annually
    Higher Calmar ration idicates that investiment had perfomed better on risk adjusted bases ..
The Calmar ration should not be used in isolation...

## Considerations
- These ratios are powerful but have limitations
- Sharpe assumes normal distributions, sortino focuses on downside, and Calmar is sensitive to drawdown period.
## Holistic Evaluation
* Use sharpe, sortino and Calmar rations togther.
* Avoid using ratins in isolation, consider various approaches of perfomance.
## Conclusion
* Ratios add in comparing risk-adjusted perfomance.
* Use them as part of a comprehensive evaluation framework.
* Ensures  historical data aligns with investiment duration for accurate assessment.

## key Takeaway
* Backtest evaluation is a multi-dimensional process, requiring a balanced approach with various metrics.
* Consider limitations and apply ratios judiciously in strategy assessment.
